In [60]:
from functions import nucl_search, save_esearch_results, plain_to_fasta

In [61]:
query = "(Drosophilidae[ORGN] NOT Drosophila melanogaster[ORGN]) AND (chromosome X[WORD] NOT PREDICTED[WORD] NOT gene[WORD]) AND 15000000:75000000[SLEN]"

In [62]:
id_list_test = nucl_search(query)

In [63]:
save_esearch_results(id_list_test[:3], "Drosophilidae")

AE - NCBI - Genome project
AJ - EBI - Direct submissions
NM - curated RefSeq

https://www.alliancegenome.org/gene/FB:FBgn0003321

Sequence Details (Mode: cDNA)

---

1. Найти референсные последовательности для sbr - https://www.alliancegenome.org/gene/FB:FBgn0003321 (Sequence Details (Mode: cDNA)):
   - найти координаты гены на X хромосоме
   - полная последовательность гена
   - сплайсированный вариант
   - консервативная кассета
2. Написать срипт, который:
   - ищет ген sbr / nxf1 в базе данных NCBI для семейства Drosophilidae
   - выбирает из найденных только адекватные варианты и формирует из них БД для бласта
3. Сделать бласт вариантов из 1-го пункта на варианты из второго пункта

In [42]:
path_to_plain_dir = "/home/artemvaska/Master_degree/Diploma/References/"

In [ ]:
plain_to_fasta(path_to_plain_dir + "sbr_RA_gene_plain.fa", fasta_line_length=80, uppercase=True)

In [45]:
plain_to_fasta(path_to_plain_dir + "sbr_RA_CDS_plain.fa", fasta_line_length=80, uppercase=True)

In [52]:
SBR_CONSERVATIVE_CASSETTE = 'ATACCATCTTTGGCCCACCTTGTAGTGCTTCGCAATCTGTCCATCTTGGAACTCGTTTTAAAGAACAATCCCTGCCGTTCCCGCTACAAGGATTCCCAGCAGTTTATCAGGTATACTATGGTGTGATTCGTTGATCTTCTAACTTTTCTAGTGGATGCTGCCGCATATTCGACGTGGGATTTGTCGCGTCTGGAGTTACAGAGGAATTAGGATTACCACATTTCCATTGTTCTGTTGTCTCCATTCCTTTTCTTTGAAGCAAATGCGCTGCAGAATTGGGGACCAAGGTCAGGTGTTGCCTTCGTCCAAAGTTTAAAGCAGCTGAGCAGGCAAAGGCATTTGGGAAATATTTTCCTTATCAGCTGTCTTAAAGCCAGGACAGATCGTATGCAGCGGAATGGATTTTGGGCAGAGCAAAAAAAAAAAAAAGAAGAAAGAAAAAAAAAGATTTGAAGTTAAAAAAAAAAAAGAAAAAAAAAGAAGAAAAAAAAAGAAAAAAAAGTACAAGCAAAAAAGGCCCAAGCCAATTGCAGTAAACTTTCGACGATGTGGCAACGCTAAACTAAGAACGCAACTGTACTTTTGACACACACACACCGGCACCGTTCGTACTTCCAATTGAGCAGGCGGACGTGCAGCAGTCCCAAAGCGAATCACATTAACGCATTAACATCGACAAATATAAATTATTTTACGAATAACAAATAATCCACCGAACCCACCCACCGCGCACCCCGCTACAACACACACACGGGCGCACGCATGTAGCCACTCCACGCCAGCGCACGCACGCATGCGCATGCGCATATGCAGTTGTGTACGTGCCACATCTAATGCTGCGTGCCCTGTGCGCGCCGTTGCGCCTTGCATTGTCGTGTCGCCGCTTGTTGATTTTGCGGTGGTTGCCAAGTAGTGGAGGCAGAGTGGAGTGCATAGGCTCACCATTCGGGTGCGCAGCGTCCAGCAGGAAGCAGAAACGACTACCAAGACGAAGGACACACCACTGACTGACTCACGTGCTAACGGACATTCTGCTGGCGCCAGCGGCCGGCGCGTTGTTTTGCCAGTCGATTACAATTTACAATTAGTATTTTCGGCTTTGTTGGGCGCATGAGCCGCCTGCGGATGCGACGTGCTTGAACATGACGAGACGAACAGAAGTTACACAGCAGTCAGCTTTGAAAGCCACTTTTTGCTTGTGGCCACTGAAAGAAAAAACAAAAAAAAAAAAATGAAAAAAAGGAAAAGAAAAAAGTGTTGATAAAAAAAAAAAAACAAAAAATACAAAAGAGAAAATACCACAAAAAAGATGCCCCGGCACAATGCACTGCTCAATTGTACTTTTCCGCTGTACTTTTCACATCGAACTATTGAAGGCACAACAATGGCCAAGCACCTAGTGCCTCTGCCTCTTCAGCTGATCAAAGCTCCGCCAATTTGTGTGTGAGCCGGTCGAGTAACGGTGAAGCTATATCAAAGATACAAACATATATACTCCGCGTGAGAAGAAGAAGGAGTCACATCTGGCCACTGACTAATCTGCAATTCCTGCAGCGCATACATCCTAAACCTCCCCCCCCCCCCACCAACGCACAAATTTAAAAAAAAAACCCCCATTTAATTATTATTTCTCGCTATTTAACTCGCATATCTATCATCGCCTCCCTGTGCTAATGACAACAAAAATACGGCCGAATGTCCACTATTTACAGCGAAGTACGTCGCAAGTTTCCCAAACTGGTTAAGTTG'

In [55]:
SBR_CASSETTE_INTRON = 'GTATACTATGGTGTGATTCGTTGATCTTCTAACTTTTCTAGTGGATGCTGCCGCATATTCGACGTGGGATTTGTCGCGTCTGGAGTTACAGAGGAATTAGGATTACCACATTTCCATTGTTCTGTTGTCTCCATTCCTTTTCTTTGAAGCAAATGCGCTGCAGAATTGGGGACCAAGGTCAGGTGTTGCCTTCGTCCAAAGTTTAAAGCAGCTGAGCAGGCAAAGGCATTTGGGAAATATTTTCCTTATCAGCTGTCTTAAAGCCAGGACAGATCGTATGCAGCGGAATGGATTTTGGGCAGAGCAAAAAAAAAAAAAAGAAGAAAGAAAAAAAAAGATTTGAAGTTAAAAAAAAAAAAGAAAAAAAAAGAAGAAAAAAAAAGAAAAAAAAGTACAAGCAAAAAAGGCCCAAGCCAATTGCAGTAAACTTTCGACGATGTGGCAACGCTAAACTAAGAACGCAACTGTACTTTTGACACACACACACCGGCACCGTTCGTACTTCCAATTGAGCAGGCGGACGTGCAGCAGTCCCAAAGCGAATCACATTAACGCATTAACATCGACAAATATAAATTATTTTACGAATAACAAATAATCCACCGAACCCACCCACCGCGCACCCCGCTACAACACACACACGGGCGCACGCATGTAGCCACTCCACGCCAGCGCACGCACGCATGCGCATGCGCATATGCAGTTGTGTACGTGCCACATCTAATGCTGCGTGCCCTGTGCGCGCCGTTGCGCCTTGCATTGTCGTGTCGCCGCTTGTTGATTTTGCGGTGGTTGCCAAGTAGTGGAGGCAGAGTGGAGTGCATAGGCTCACCATTCGGGTGCGCAGCGTCCAGCAGGAAGCAGAAACGACTACCAAGACGAAGGACACACCACTGACTGACTCACGTGCTAACGGACATTCTGCTGGCGCCAGCGGCCGGCGCGTTGTTTTGCCAGTCGATTACAATTTACAATTAGTATTTTCGGCTTTGTTGGGCGCATGAGCCGCCTGCGGATGCGACGTGCTTGAACATGACGAGACGAACAGAAGTTACACAGCAGTCAGCTTTGAAAGCCACTTTTTGCTTGTGGCCACTGAAAGAAAAAACAAAAAAAAAAAAATGAAAAAAAGGAAAAGAAAAAAGTGTTGATAAAAAAAAAAAAACAAAAAATACAAAAGAGAAAATACCACAAAAAAGATGCCCCGGCACAATGCACTGCTCAATTGTACTTTTCCGCTGTACTTTTCACATCGAACTATTGAAGGCACAACAATGGCCAAGCACCTAGTGCCTCTGCCTCTTCAGCTGATCAAAGCTCCGCCAATTTGTGTGTGAGCCGGTCGAGTAACGGTGAAGCTATATCAAAGATACAAACATATATACTCCGCGTGAGAAGAAGAAGGAGTCACATCTGGCCACTGACTAATCTGCAATTCCTGCAGCGCATACATCCTAAACCTCCCCCCCCCCCCACCAACGCACAAATTTAAAAAAAAAACCCCCATTTAATTATTATTTCTCGCTATTTAACTCGCATATCTATCATCGCCTCCCTGTGCTAATGACAACAAAAATACGGCCGAATGTCCACTATTTACAG'

In [71]:
def read_fasta(file: str) -> dict:
    seqs = {}
    seq = []
    with open(file) as infile:
        for line in infile:
            line = line.strip()
            if line.startswith(">"):
                header = line[1:]
            else:
                seq.append(line)
    seqs[header] = "".join(seq)
    return seqs

In [75]:
sbr_gene = read_fasta("../References/sbr_RA_CDS.fa")

In [79]:
sbr_gene['sbr-RA-cds']

'ATGCCCAAACGCGGCGGTGGCAGTAGCCAGCGGTACAACAACAACGTTGGAAATGGCGGCGGACGTTACAACGCTCCCGAGGATTTCGATGATTTTGGTCATGTGGAGGATCGCCAGCGACGCAAGGATCGAAACAAGCGGCGCGTCAGCTTTAAGCCCTCCCAATGTCTACATAACAAAAAGGACATCAAGCTGCGACCCGAAGATTTGCGTCGATGGGACGAGGATGATGACATGAGCGACATGACCACGGCCGTTAAGGATAGACCCACCTCCCGACGTCGGGGATCGCCCATTCCGCGCGGCAAGTTCGGCAAACTGATGCCCAACAGCTTTGGCTGGTACCAAGTCACGTTACAAAACGCCCAGATATACGAAAAGGAAACACTCTTGAGTGCTCTATTGGCAGCGATGTCGCCACATGTCTTTATTCCTCAATATTGGCGAGTGGAGCGAAACTGCGTAATCTTCTTTACGGACGACTACGAGGCAGCCGAACGCATTCAACATCTGGGCAAGAATGGCCATCTTCCAGATGGCTATCGTCTGATGCCACGAGTACGCAGCGGTATACCACTAGTGGCCATCGACGATGCCTTCAAGGAGAAGATGAAGGTCACAATGGCCAAGCGTTACAATATTCAAACCAAGGCGCTGGATCTTTCCCGTTTTCATGCAGATCCGGATCTTAAGCAAGTTTTCTGCCCACTCTTTCGTCAGAATGTGATGGGCGCTGCCATTGACATTATGTGCGACAATATACCCGATTTGGAGGCACTTAACCTGAATGACAACTCCATTAGCAGCATGGAGGCGTTTAAGGGTGTGGAGAAACGCTTACCGAACCTCAAGATTCTCTATTTGGGGGATAACAAGATACCATCTTTGGCCCACCTTGTAGTGCTTCGCAATCTGTCCATCTTGGAACTCGTTTTAAAGAACAATCCCTGCCGTTCCCGCTACAAGGATTCCCAGCAGTTTATCAGCGAAGTACGTCGC

https://biopython.org/docs/dev/Tutorial/chapter_blast.html

In [1]:
from Bio import Blast

In [2]:
Blast.tool

'biopython'

In [3]:
Blast.email = "artem_vasilev_01@list.ru"

In [4]:
fasta_string = open("../References/sbr_RA_gene.fa").read()

In [ ]:
"all [filter] NOT(environmental samples[organism] OR metagenomes[orgn]) AND txid3702[ORGN] AND txid9606[ORGN]"

In [67]:
result_stream = Blast.qblast(program="blastn",
                             database="wgs",
                             sequence=fasta_string,
                             entrez_query="Drosophilidae AND NOT Drosophila melanogaster",
                             megablast=True)

Проблемы: 

- параметр database м.б. только "nt"
- не удается сделать exclude в entrez_query

Решение:

- cmd tool blast+ с кастомной базой данных
- cmd blast+ -remote
- просто ncbi blastn

Drosophilidae (taxid:7214)

Drosophila melanogaster (taxid:7227)

https://biochem.slu.edu/bchm628/handouts/2013/Entrez_boolian_searches.pdf

In [ ]:
with open("sbr_RA_gene_blast.xml", "wb") as out_stream:
    out_stream.write(result_stream.read())
result_stream.close()

result_stream = open("sbr_RA_gene_blast.xml", "rb")

blast_record = Blast.read(result_stream)

In [77]:
blast_record

<Bio.Blast.Record query.id='Query_6742799'; no hits>

In [78]:
print(blast_record)

Program: BLASTN 2.16.0+
     db: wgs
  Query: Query_6742799 (length=14341)
         sbr-RA-gene
   Hits: No hits found


Установка blast+ на linux (https://www.ncbi.nlm.nih.gov/books/NBK52640/):

1. https://ftp.ncbi.nlm.nih.gov/blast/executables/LATEST/ (ncbi-blast-2.16.0+-x64-linux.tar.gz)
2. `mv ~/Downloads/ncbi-blast-2.16.0+-x64-linux.tar.gz ~`
3. `tar zxvpf ncbi-blast-2.16.0+-x64-linux.tar.gz`
4. `export PATH=$PATH:$HOME/ncbi-blast-2.16.0+/bin`
5. `mkdir $HOME/blastdb`
6. `export BLASTDB=$HOME/blastdb`
7. export into bash_profile (`nano .bash_profile`)

blastn -db nt -query References/sbr_RA_gene.fa -out References/test.out -remote

Available DB in NCBI blastn -remote:

https://rc.dartmouth.edu/index.php/blast-introduction/blast-databases/

blastn -db nt -query References/sbr_RA_gene.fa -out References/test3.out -remote -entrez_query "NOT Drosophila melanogaster AND Drosophilidae"

Попробовать распарсить XML результат бласта, полученный 1) через cmd, 2) через веб-сервис NCBI

Попробовать скачать wgs db локально